In [1]:
import dgl
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt               

In [2]:
dataset = dgl.data.CSVDataset('../../Dati/Dataset_otto_classi/')
dataset

Done loading data from cached files.


Dataset("Dataset_otto_classi", num_graphs=2276, save_path=../../Dati/Dataset_otto_classi/Dataset_otto_classi)

In [3]:
numer_class = []
a = 0
b = 0 
c = 0
d = 0
e = 0
f = 0
gg = 0
h = 0
for g, i in dataset:
    if i["label"].item() not in numer_class:
        numer_class.append(i["label"].item())
    if i["label"].item() == 0:
        a += 1
    elif i["label"].item() == 1:
        b += 1
    elif i["label"].item() == 2:
        c += 1
    elif i["label"].item() == 3:
        d += 1
    elif i["label"].item() == 4:
        e += 1
    elif i["label"].item() == 5:
        f += 1
    elif i["label"].item() == 6:
        gg += 1
    elif i["label"].item() == 7:
        h += 1

print(a, b, c, d, e, f, gg, h, (a+b+c+d+e+f+gg+h))
print(len(numer_class))

293 284 295 298 294 291 221 300 2276
8


In [4]:
dim_nfeats = 2
gclasses = len(numer_class)

In [5]:
size_batch = len(numer_class)*2
size_batch

16

In [6]:
from dgl.dataloading import GraphDataLoader
from torch.utils.data.sampler import SubsetRandomSampler

num_examples = len(dataset)
num_train = int(num_examples * 0.8)

train_sampler = SubsetRandomSampler(torch.arange(num_train))
test_sampler = SubsetRandomSampler(torch.arange(num_train, num_examples))

train_dataloader = GraphDataLoader(
    dataset, sampler=train_sampler, batch_size= size_batch, drop_last=False)
test_dataloader = GraphDataLoader(
    dataset, sampler=test_sampler, batch_size= size_batch, drop_last=False)

In [7]:
it = iter(train_dataloader)
batch = next(it)
print(batch)

[Graph(num_nodes=3370, num_edges=123604,
      ndata_schemes={'label': Scheme(shape=(2,), dtype=torch.float32)}
      edata_schemes={}), {'label': tensor([7., 1., 7., 0., 3., 0., 2., 4., 1., 4., 6., 4., 7., 2., 3., 4.])}]


In [8]:
batched_graph, labels = batch
print('Number of nodes for each graph element in the batch:', batched_graph.batch_num_nodes())
print('Number of edges for each graph element in the batch:', batched_graph.batch_num_edges())

# Recover the original graph elements from the minibatch
graphs = dgl.unbatch(batched_graph)
print('The original graphs in the minibatch:')
print(graphs)

Number of nodes for each graph element in the batch: tensor([152, 219,  21, 141, 129, 396, 246, 358, 119, 357,  30, 357, 149, 247,
         96, 353])
Number of edges for each graph element in the batch: tensor([ 4670,  6952,   264,  4540,  4352, 14374, 10526, 14518,  3876, 13782,
          662, 14494,  4450,  9232,  2796, 14116])
The original graphs in the minibatch:
[Graph(num_nodes=152, num_edges=4670,
      ndata_schemes={'label': Scheme(shape=(2,), dtype=torch.float32)}
      edata_schemes={}), Graph(num_nodes=219, num_edges=6952,
      ndata_schemes={'label': Scheme(shape=(2,), dtype=torch.float32)}
      edata_schemes={}), Graph(num_nodes=21, num_edges=264,
      ndata_schemes={'label': Scheme(shape=(2,), dtype=torch.float32)}
      edata_schemes={}), Graph(num_nodes=141, num_edges=4540,
      ndata_schemes={'label': Scheme(shape=(2,), dtype=torch.float32)}
      edata_schemes={}), Graph(num_nodes=129, num_edges=4352,
      ndata_schemes={'label': Scheme(shape=(2,), dtype=torch.f

In [9]:
from dgl.nn import GraphConv

class GCN(nn.Module):
    def __init__(self, in_feats, h_feats, num_classes):
        super(GCN, self).__init__()
        self.conv1 = GraphConv(in_feats, h_feats)
        self.conv2 = GraphConv(h_feats, num_classes)

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        g.ndata['h'] = h
        return dgl.mean_nodes(g, 'h')

In [10]:
if gclasses == 2:
    num_of_epoches = 200
elif gclasses >= 2:
    num_of_epoches = 1000

In [ ]:
# Create the model with given dimensions
loss_scores = {}
controllo = []
min_valore = 0.45

model = GCN(dim_nfeats, 32, gclasses)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)



for epoch in range(num_of_epoches):
    print("Epoca: ", epoch)
    for batched_graph, labels in train_dataloader:
        pred = model(batched_graph, batched_graph.ndata['label'].float())
        loss = F.cross_entropy(pred, labels['label'].long())
        #print("epoca: ", epoch, loss)
        optimizer.zero_grad()
        #print(optimizer)
        loss.backward()
        #print(loss)
        optimizer.step()
        #print(optimizer)
    print("   Loss -> ", loss.item())
    loss_scores[epoch] = loss.item()
    """if loss.item() < 0.4 and epoch >= 400:
        break"""
    if loss.item() < min_valore:
        controllo.append((epoch, loss.item()))
    

Epoca:  0


/Applications/anaconda3/lib/python3.8/site-packages/torch/autocast_mode.py:162: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


   Loss ->  2.0351972579956055
Epoca:  1
   Loss ->  2.059483289718628
Epoca:  2
   Loss ->  2.1311631202697754
Epoca:  3
   Loss ->  1.9834623336791992
Epoca:  4
   Loss ->  2.0626723766326904
Epoca:  5
   Loss ->  1.9724845886230469
Epoca:  6
   Loss ->  2.039252996444702
Epoca:  7
   Loss ->  2.2132976055145264
Epoca:  8
   Loss ->  2.0578413009643555
Epoca:  9
   Loss ->  1.8371858596801758
Epoca:  10
   Loss ->  1.9486955404281616
Epoca:  11
   Loss ->  1.7760030031204224
Epoca:  12
   Loss ->  2.0213472843170166
Epoca:  13
   Loss ->  2.0525076389312744
Epoca:  14
   Loss ->  1.8263322114944458
Epoca:  15
   Loss ->  1.880799412727356
Epoca:  16
   Loss ->  1.903599739074707
Epoca:  17
   Loss ->  1.9072200059890747
Epoca:  18
   Loss ->  1.7796670198440552
Epoca:  19
   Loss ->  1.9025520086288452
Epoca:  20
   Loss ->  1.9603352546691895
Epoca:  21
   Loss ->  1.8550127744674683
Epoca:  22
   Loss ->  2.500009775161743
Epoca:  23
   Loss ->  1.6959277391433716
Epoca:  24
   Los

   Loss ->  1.4507588148117065
Epoca:  196
   Loss ->  1.716163992881775
Epoca:  197
   Loss ->  1.6424177885055542
Epoca:  198
   Loss ->  1.4268208742141724
Epoca:  199
   Loss ->  1.0991170406341553
Epoca:  200
   Loss ->  1.9242924451828003
Epoca:  201
   Loss ->  1.5942527055740356
Epoca:  202
   Loss ->  1.573809027671814
Epoca:  203
   Loss ->  1.4590204954147339
Epoca:  204
   Loss ->  1.3777894973754883
Epoca:  205
   Loss ->  1.8011728525161743
Epoca:  206
   Loss ->  1.4379082918167114
Epoca:  207
   Loss ->  1.8887468576431274
Epoca:  208
   Loss ->  1.3144489526748657
Epoca:  209
   Loss ->  1.942773461341858
Epoca:  210
   Loss ->  1.7669014930725098
Epoca:  211
   Loss ->  1.625494360923767
Epoca:  212
   Loss ->  1.5117530822753906
Epoca:  213
   Loss ->  1.654292106628418
Epoca:  214
   Loss ->  1.9323176145553589
Epoca:  215
   Loss ->  1.5588546991348267
Epoca:  216
   Loss ->  1.4155546426773071
Epoca:  217
   Loss ->  2.191129446029663
Epoca:  218
   Loss ->  1.626

   Loss ->  1.6123566627502441
Epoca:  388
   Loss ->  1.9274078607559204
Epoca:  389
   Loss ->  1.742551326751709
Epoca:  390
   Loss ->  1.169581651687622
Epoca:  391
   Loss ->  1.6280401945114136
Epoca:  392
   Loss ->  1.9536875486373901
Epoca:  393
   Loss ->  1.420792579650879
Epoca:  394
   Loss ->  1.4157453775405884
Epoca:  395
   Loss ->  1.8429670333862305
Epoca:  396
   Loss ->  2.172255277633667
Epoca:  397
   Loss ->  1.8037012815475464
Epoca:  398
   Loss ->  1.6808017492294312
Epoca:  399
   Loss ->  1.4060920476913452
Epoca:  400
   Loss ->  1.3370901346206665
Epoca:  401
   Loss ->  1.490869164466858
Epoca:  402
   Loss ->  1.2771152257919312
Epoca:  403
   Loss ->  1.7357844114303589
Epoca:  404
   Loss ->  1.4957410097122192
Epoca:  405
   Loss ->  1.461891531944275
Epoca:  406
   Loss ->  1.80711030960083
Epoca:  407
   Loss ->  1.634230613708496
Epoca:  408
   Loss ->  1.096903681755066
Epoca:  409
   Loss ->  1.4869078397750854
Epoca:  410
   Loss ->  1.6246701

   Loss ->  2.0447380542755127
Epoca:  581
   Loss ->  1.9445627927780151
Epoca:  582
   Loss ->  1.3252583742141724
Epoca:  583
   Loss ->  1.527915120124817
Epoca:  584
   Loss ->  1.5964432954788208
Epoca:  585
   Loss ->  1.7908462285995483
Epoca:  586
   Loss ->  1.8194116353988647
Epoca:  587
   Loss ->  1.7059717178344727
Epoca:  588
   Loss ->  1.3940467834472656
Epoca:  589
   Loss ->  1.3638819456100464
Epoca:  590
   Loss ->  1.9197043180465698
Epoca:  591
   Loss ->  1.5798282623291016
Epoca:  592
   Loss ->  1.228169560432434
Epoca:  593
   Loss ->  1.3823970556259155
Epoca:  594
   Loss ->  1.3093427419662476
Epoca:  595
   Loss ->  1.270687222480774
Epoca:  596
   Loss ->  1.4971381425857544
Epoca:  597
   Loss ->  2.2062087059020996
Epoca:  598
   Loss ->  1.9066404104232788
Epoca:  599
   Loss ->  1.839554786682129
Epoca:  600
   Loss ->  1.6472550630569458
Epoca:  601
   Loss ->  1.7141532897949219
Epoca:  602
   Loss ->  0.959510326385498
Epoca:  603
   Loss ->  1.61

   Loss ->  1.0198644399642944
Epoca:  773
   Loss ->  1.882541298866272
Epoca:  774
   Loss ->  1.998144507408142
Epoca:  775
   Loss ->  1.4560465812683105
Epoca:  776
   Loss ->  1.8318229913711548
Epoca:  777
   Loss ->  1.9031764268875122
Epoca:  778
   Loss ->  1.2594881057739258
Epoca:  779
   Loss ->  1.6624094247817993
Epoca:  780
   Loss ->  1.9050666093826294
Epoca:  781
   Loss ->  1.7530049085617065
Epoca:  782
   Loss ->  1.8544703722000122
Epoca:  783
   Loss ->  1.5427130460739136
Epoca:  784
   Loss ->  1.2996363639831543
Epoca:  785
   Loss ->  1.3828204870224
Epoca:  786
   Loss ->  1.572743535041809
Epoca:  787
   Loss ->  1.2051799297332764
Epoca:  788
   Loss ->  1.1123014688491821
Epoca:  789
   Loss ->  1.7932778596878052
Epoca:  790
   Loss ->  1.4580353498458862
Epoca:  791
   Loss ->  1.9328327178955078
Epoca:  792
   Loss ->  1.8422914743423462
Epoca:  793
   Loss ->  1.5286178588867188
Epoca:  794
   Loss ->  1.5229068994522095
Epoca:  795
   Loss ->  1.737

In [ ]:
print("Ultmia epoca ", epoch," con valore di loss ->", loss.item())

In [ ]:
for x in controllo:
    if x[1] < 0.45:
        print(x)

In [ ]:
plt.figure(figsize = (20,10))
plt.plot(loss_scores.values())
plt.xticks(range(0,num_of_epoches,1), loss_scores.keys())
plt.title("Loss Graphic")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.show()

In [ ]:
num_correct = 0
num_tests = 0
for batched_graph, labels in test_dataloader:
    pred = model(batched_graph, batched_graph.ndata['label'].float())
    num_correct += (pred.argmax(1) == labels['label'].long()).sum().item()
    num_tests += len(labels['label'].long())
    
    
print('Test accuracy:', num_correct / num_tests)

## Analisi risultati
Con il dataset Binario l'accuratezza si aggira attorno al 0.9.
Con il dataset Multiplo il quale momentaneamente contiene 4 classi di grafi l'accuratezza si aggira attorno al 0.54.

## PROVE


Numero di nodi 16

batch = 10 epoche 1000 con soglia 400epoche e loss < 0.4 -> ***ACCURATEZZA OTTENUTA*** 0,61

batch = 8 epoche 5000  -> ***ACCURATEZZA OTTENUTA*** 0,61 con ultima loss -> 0.7

batch = 8 epoche 1000  -> ***ACCURATEZZA OTTENUTA*** 0,59 con ultima loss -> 0.87

batch = 8 epoche 1000  -> ***ACCURATEZZA OTTENUTA*** 0,63 con ultima loss -> 1.61





Numero di nodi 32

batch = 8 epoche 1000 ->  ***ACCURATEZZA OTTENUTA*** 0,62 con ultima loss ->  1.09


Numero di nodi 12

batch = 8 epoche 1000 ->  ***ACCURATEZZA OTTENUTA*** 0,62 con ultima loss ->   0.75







Numero di nodi 12 classificazione di sei classi 

batch = 12 epoche 1000 -> ***ACCURATEZZA OTTENUTA*** 0,42 con ultima loss ->  0.84